In [1]:
import requests 
import json 
import pprint
from datetime import date


In [2]:
class Report: 
    def __init__(self):
        
        self.url ="https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest" #link per i report
        self.params = {
            'start': '1', #inizia dalla prima cripto in classifica
            'limit': '100',  #finisce con la 100'esima cripto in classifca
            'convert': "USD"  #valuta delle cripto -> dollaro 
        }
        
        self.headers = {
            'Accepts': 'application/json', #formato della risposta 
            'X-CMC_PRO_API_KEY': 'e5c4253e-a74e-4274-84a0-9ed17d09b04f' #chiave per poter scaricare i dati 
        }
        
    def fetchCurrentData(self): 
        r = requests.get(url = self.url, headers = self.headers, params = self.params).json() #fa la richieste per i dati 
        return r['data'] #restituisce i dati in formato json 


In [3]:
report = Report()

# currencies = report.fetchCurrentData()
# with open("output.json", "w") as outfile: 
#     json.dump(currencies, outfile, indent = 4)

data = json.load(open("old_data.json"))


In [4]:
def max_volume(data): #trova la valuta con maggiore volume delle ultime 24h
    volumes = {v['quote']['USD']['volume_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    return volumes[max(volumes)] #ritorna nome della cripto con volume maggiore


In [5]:
def best_and_worst_criptos(data): #trova le migliori e le peggiori 10 criptovalute (per incremento in percentuale nelle ultime 24 ore)

    volumes = { v['quote']['USD']['percent_change_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    sorted_volumes = sorted(volumes) #lista ordinata in base alle chiavi del dizionario
    best_volumes = sorted_volumes[:10]  #lista con i volumi migliori
    worst_volumes = sorted_volumes[-10:] #lista con i volumi peggiori
    
    best = []
    worst = []
    
    for v,name in volumes.items():  #ciclo sul dizionario 
        if v in best_volumes:  #se la chiave è nella lista dei migiori 
            best.append(name)  #salvo il nome della cripto 
        if v in worst_volumes: 
            worst.append(name)
            
    return best, worst





In [6]:
def first_20(data): #trova la quantità di denaro necessaria per acquistare una unità di ciascuna delle prime 20 criptovalute
    volumes = {v['name']: v['quote']['USD']['price'] for v in data[:20]} #dizionario con chiave nome e valore
    return volumes



In [7]:
def find_criptos(data, volume): 
    #trova la quantità di denaro necessaria per aqustare una unità di tutte le cripto
    #il cui volume delle ultime 24 h sia superiore a 'volume'
    
    volumes = {v['name']: v['quote']['USD']['price'] for v in data
                if v['quote']['USD']['volume_24h'] > volume}
    
    return volumes


In [8]:
#trova la precentuale di guadagno o perdita che ho se comprassi una unità di ciascuna
#delle prime 20 criptovalute il giorno prima (ipotizzando che la classifica non sia cambiata)
def find_percentage(new_data, old_data):
    new_20 = first_20(new_data)
    old_20 = first_20(old_data)
    
    #creo dizionario con nome e precentuale du guadagno o peridata
    r = {f : (new_20[f] - old_20[i]) / old_20[f] *100 for f,i in zip(new_20,old_20)}
    return r
    

In [9]:
today = date.today() #data di oggi 
new_data = report.fetchCurrentData() #scarica i dati dai Coinmarket
old_data = json.load(open("old_data.json")) #prendo un file vecchio

max_volume_cripto = max_volume (data)
bests, worsts = best_and_worst_criptos(data) 
f_20 = first_20(data)
criptos = find_criptos(data, 76000000)
percentage = find_percentage(new_data, old_data)

result = {'max_volume_cripto': max_volume_cripto, 
            'bests': bests,
            'worst': worsts,
            'first_20_criptos': f_20,
             'criptos': criptos,
            'percentage': percentage}

#scrivo l'output sul file con la data di oggi 
with open(f"{today}.json", "w") as outfile: 
     json.dump(result, outfile, indent = 4)

KeyError: 'VeChain'